In [3]:
%pip install pandas scikit-learn fairlearn numpy

     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 60.8/60.8 kB 3.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.6 MB ? eta -:--:--
   - -------------------------------------- 0.4/11.6 MB 12.9 MB/s eta 0:00:01
   --- ------------------------------------ 1.1/11.6 MB 18.3 MB/s eta 0:00:01
   --------------- ------------------------ 4.4/11.6 MB 40.3 MB/s eta 0:00:01
   --------------------- ------------------ 6.3/11.6 MB 50.4 MB/s eta 0:00:01
   ----------------------------------- ---- 10.3/11.6 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------  11.6/11.6 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------  11.6/11.6 MB 93.9 MB/s eta 0:00:01
   ---------------------------------------- 11.6/11.6 MB 54.7 MB/s eta 0:00:0

ERROR: Could not install packages due to an OSError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\dz28l\\AppData\\Local\\Packages\\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\\LocalCache\\local-packages\\Python311\\site-packages\\scipy\\ndimage\\_ni_docstrings.py'
Check the permissions.


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\dz28l\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


^C
Note: you may need to restart the kernel to use updated packages.


  Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl.metadata (19 kB)
  Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl.metadata (13 kB)
  Using cached fairlearn-0.11.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl.metadata (60 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached pandas-2.2.3-cp311-cp311-win_amd64.whl (11.6 MB)
Using cached scikit_learn-1.5.2-cp311-cp311-win_amd64.whl (11.0 MB)
Using cached fairlearn-0.11.0-py3-none-any.whl (232 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached scipy-1.14.1-cp311-cp311-win_amd64.whl (44.8 MB)



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: C:\Users\dz28l\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from fairlearn.metrics import MetricFrame, selection_rate, demographic_parity_difference
from fairlearn.postprocessing import ThresholdOptimizer

# Step 1: Load the Data
file_path = 'data/tcc_ceds_music.csv'  # Replace with your file path if needed
data = pd.read_csv(file_path)

# Step 2: Preprocess Data
# Select relevant features based on columns in your dataset
features = [
    'len', 'dating', 'violence', 'world/life', 'night/time', 'shake the audience',
    'family/gospel', 'romantic', 'communication', 'obscene', 'music', 'movement/places',
    'light/visual perceptions', 'family/spiritual', 'like/girls', 'sadness', 'feelings',
    'danceability', 'loudness', 'acousticness', 'instrumentalness', 'valence', 'energy'
]
target = 'topic'  # Assuming the 'topic' column is our label for recommendation

# Encode the target variable (topic)
data[target] = LabelEncoder().fit_transform(data[target])

# Step 3: Split Data
# Using 'age' as a proxy for the sensitive attribute here
X_train, X_test, y_train, y_test, sensitive_train, sensitive_test = train_test_split(
    data[features], data[target], data['age'], test_size=0.2, random_state=42
)

# Step 4: Define and Train the Model
model = KNeighborsClassifier(n_neighbors=5)
model.fit(X_train, y_train)

# Step 5: Fairness Evaluation
y_pred = model.predict(X_test)

# Using Fairlearn to assess fairness
metric_frame = MetricFrame(
    metrics={"accuracy": lambda y_true, y_pred: (y_true == y_pred).mean(),
             "selection_rate": selection_rate},
    y_true=y_test,
    y_pred=y_pred,
    sensitive_features=sensitive_test
)

# Display metrics for each group
print("Accuracy per group (based on age):", metric_frame.by_group["accuracy"])
print("Selection rate per group (based on age):", metric_frame.by_group["selection_rate"])
print("Demographic Parity Difference (based on age):", demographic_parity_difference(y_true=y_test, y_pred=y_pred, sensitive_features=sensitive_test))


Accuracy per group (based on age): age
0.014286    0.894118
0.028571    0.852713
0.042857    0.840000
0.057143    0.809917
0.071429    0.880734
              ...   
0.942857    0.850000
0.957143    0.888889
0.971429    0.777778
0.985714    0.888889
1.000000    0.727273
Name: accuracy, Length: 70, dtype: float64
Selection rate per group (based on age): age
0.014286    0.023529
0.028571    0.015504
0.042857    0.046667
0.057143    0.033058
0.071429    0.036697
              ...   
0.942857    0.150000
0.957143    0.333333
0.971429    0.000000
0.985714    0.222222
1.000000    0.000000
Name: selection_rate, Length: 70, dtype: float64
Demographic Parity Difference (based on age): 0.3333333333333333


In [ ]:
gender = pd.read_csv('gender.csv')

In [ ]:
gender[gender['NAME'].str.lower() == "les paul".lower()]